In [1]:
using DrWatson

@quickactivate "GRNEvoContingency"

projectname()

"GRNEvoContingency"

In [2]:
include(srcdir("TissueModel_ND.jl"))
include(srcdir("Evolution.jl"))
include(srcdir("FitnessFunctions.jl"))

include(srcdir("NetworkTopologies.jl"))
include(srcdir("Utility.jl"))

using Plots
using StatsPlots
using Base.Threads
using Base.Threads: @spawn

In [ ]:
Threads.nthreads()

In [ ]:
function random_start_network(topology)
    (0.9995 .^ rand(0:10000,Ng,Ng+1)) .* 10 .* rand(Ng,Ng+1) .* topology
end

problem_types = Dict("feed_forward" => w_feed_forward,
                    "mutual_inh" => w_mutual_inh,
                    "frozen_osc" => w_frozen_osc,
                    "overlap_dom" => w_overlap_dom,
                    "bistable"=> w_bistable,
                    "classical" => w_classical);


Define start network and default model parameters:

In [ ]:
grn_parameters = DefaultGRNParameters();

# n_stripe = 1

# stripe_threshold = 5.

# target = generate_stripe_distribution([50.],[10.])

# output_gene = 3

# fitness_function = s -> fitness_evaluation(s,x->f_sim(x,stripe_threshold,n_stripe,target),output_gene);

n_stripe = 1

stripe_threshold = 5.

min_width = 5.

target = [30.,40.,30.]

output_gene = 3

fitness_function = s -> fitness_evaluation(s,x->f_sim(x,stripe_threshold,n_stripe,target,min_width),output_gene);


In [ ]:
β = Inf

max_gen = 10000

tolerance = 4.

In [ ]:
n_traj = 40

results = Dict("feed_forward" => Array{Any}(nothing,n_traj),
                    "mutual_inh" => Array{Any}(nothing,n_traj),
                    "frozen_osc" => Array{Any}(nothing,n_traj),
                    "overlap_dom" => Array{Any}(nothing,n_traj),
                    "bistable"=> Array{Any}(nothing,n_traj),
                    "classical" => Array{Any}(nothing,n_traj));

In [ ]:
start_topology = w_frozen_osc
# start_topology = rand([-1,1],(3,4))
# start_topology[2,4] = 0.
# start_topology[3,4] = 0.

viable_mutations = ones(Ng,Ng+1)
viable_mutations[2,4] = 0.
viable_mutations[3,4] = 0.

mutation_op = MutationOperator(Normal,(μ = 0.0,σ = 1.),start_topology)

noise_application = (x,n) -> x + n*x 

mutate_function = i -> noise(i,mutation_op,noise_application);

start_network = random_start_network(start_topology)

In [ ]:
p_final,evo_trace = SSWM_Evolution(start_network,grn_parameters,β,max_gen,tolerance,fitness_function,mutate_function);

In [ ]:
sum(map(x->x == :MaxIters,evo_trace.retcodes))

In [ ]:
plot(map(x->x[1],evo_trace.fitness_trajectory))

In [ ]:
plot(map(x->x[2],evo_trace.fitness_trajectory))

In [ ]:
plot_SS(p_final.dominant_individual.phenotype.u[end])

In [ ]:
sum(map(x->x== ReturnCode.MaxIters,evo_trace.retcodes))

In [ ]:
plot(p_final.dominant_individual.phenotype.u[end][3,:])

In [ ]:
f_sim(p_final.dominant_individual.phenotype.u[end][3,:],stripe_threshold,n_stripe,target,min_width)

In [ ]:
p_final.dominant_individual.genotype.p[1]

# 0.0        0.0     0.0       5.53168
# -0.806399   0.0     0.0       0.0
#  0.395937  -3.7749  0.306592  0.0 hill no mult

# 0.819043  -8.16315   -6.62008   -3.56115
# -3.73685   -4.77473   -0.909651   0.0
# -5.36291   -0.462706   1.05325    0.0 hill mult

In [ ]:
evo_trace.retcodes

In [ ]:
w = [   2.32579   -4.62089   1.13037  -2.35557;
0.846838  -2.31798  -2.89189   0.0;
-1.38083    5.75874  -8.84447   0.0]

In [ ]:
p = (w,grn_parameters.degradation)
    
grn = ODEProblem(gene_regulation_1d!,grn_parameters.g0,(0,Inf),p)

development = DefaultGRNSolver()

founder = Individual(grn,development);

In [ ]:
plot(founder.phenotype.u[end][1,:])
plot!(founder.phenotype.u[end][2,:])
plot!(founder.phenotype.u[end][3,:])

In [ ]:
@benchmark Individual(grn,development)

In [ ]:
for (problem, start_topology) in problem_types

    @printf "Topology: %s " problem
    @printf "\n" 

    mutation_op = MutationOperator(Normal,(μ = 0.0,σ = 1.),abs.(start_topology))

    noise_application = (x,n) -> x + n 

    mutate_function = i -> noise(i,mutation_op,noise_application);

    start_network = random_start_network(start_topology)

    @sync for t in 1:n_traj
        @spawn results[problem][t] = SSWM_Evolution(start_network,grn_parameters,β,max_gen,tolerance,fitness_function,mutate_function)[1]
    end

end

In [ ]:
function find_best(results)

    out = Dict("feed_forward" => (1,0.0),
                    "mutual_inh" =>  (1,0.0),
                    "frozen_osc" =>  (1,0.0),
                    "overlap_dom" =>  (1,0.0),
                    "bistable"=>  (1,0.0));

    for (p,_) in out

        r = results[p]

        idmax = findmin(map(x->x.fitness[2],r))[2]

        out[p] = (idmax,r[idmax].fitness[2])
    end

    return out

end

In [ ]:
find_best(results)

In [ ]:
findmin(x->x.fitness[2],results["feed_forward"])

Plot convergence of first fitness component (n_stripes):

In [ ]:
plot(map(x->x[1],evo_trace.fitness_trajectory))

Plot convergence of second fitness component (equalness):

In [ ]:
plot(map(x->x[2],evo_trace.fitness_trajectory))

Check out phenotype:

In [ ]:
plot_SS(p_final.dominant_individual.phenotype.u[end])

In [ ]:
plot(p_final.dominant_individual.phenotype.u[end][3,:])

In [ ]:
p_final.fitness

In [ ]:
p_final.dominant_individual.genotype.p[1]

Run evolution for an extended period, using previous output as start population:

In [ ]:
max_gen = 50000

SSWM_Evolution!(p_final,evo_trace,β,max_gen,tolerance,fitness_function,mutate_function)

In [ ]:
# frozen osc

# 2.45845  -2.18826  -0.654486  0.100882
#  0.0       0.33589   0.0       0.0
#  3.61625  -6.32603   0.0       0.0